In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
string = 'mot a trouver caca'
import re
regex = '\bcaca\b'
re.

{1, 2, 3}

In [7]:
from glob import glob
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import re
from ast import literal_eval

# to remove limits on display
pd.set_option('display.max_colwidth', -1)

In [2]:
import pyConTextNLP.pyConText as pyConText
import pyConTextNLP.itemData as itemData

def markup_sentence(s, modifiers, targets, prune_inactive=True):
    """
    """
    markup = pyConText.ConTextMarkup()
    markup.setRawText(s)
    markup.cleanText()
    markup.markItems(modifiers, mode="modifier")
    markup.markItems(targets, mode="target")
    markup.pruneMarks()
    markup.dropMarks('Exclusion')
    # apply modifiers to any targets within the modifiers scope
    markup.applyModifiers()
    markup.pruneSelfModifyingRelationships()
    if prune_inactive:
        markup.dropInactiveModifiers()
    return markup

In [3]:
modifiers = itemData.get_items(
    "https://raw.githubusercontent.com/chapmanbe/pyConTextNLP/master/KB/lexical_kb_05042016.yml")
targets = itemData.get_items(
    "https://raw.githubusercontent.com/chapmanbe/pyConTextNLP/master/KB/utah_crit.yml")


/home/yoann/.local/lib/python3.6/site-packages/pyConTextNLP/itemData.py:40: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  context_items =  [contextItem((d["Lex"],


In [6]:
sentences = [
'The patient presents no sign of pulmonary embolia',
'The uncle of the patient died of cancer three weeks ago',
'A sexual partner of the patient presents signs of hepatitis C',
]

for s in sentences:
    print(markup_sentence(s, modifiers, targets))

__________________________________________
rawText: Le patient est mort
cleanedText: Le patient est mort
__________________________________________

__________________________________________
rawText: The uncle of the patient died of cancer three weeks ago
cleanedText: The uncle of the patient died of cancer three weeks ago
********************************
TARGET: <id> 51371081368255570647537896847795691968 </id> <phrase> cancer </phrase> <category> ['cancer'] </category> 
__________________________________________

__________________________________________
rawText: A sexual partner of the patient presents signs of hepatitis C
cleanedText: A sexual partner of the patient presents signs of hepatitis C
__________________________________________



In [8]:
def get_filters(
    folder='context_java_abdaoui',
    neg_only=True,
    ):
    all_regex = []
    
    regex_files = glob(folder + '/regex*.txt')
    
    for file in regex_files:
        with open(file, 'r', encoding="utf8") as f:
            regex = f.readlines()[0]
            try:
                all_regex.extend(literal_eval(regex))
            except:
                all_regex.extend(regex.strip("\"").split('", "'))
                
    formatted_regex = [regex.replace('"', '').replace("'", '').replace(" ,", ',').rsplit(',',2) for regex in all_regex]

    regex_df = pd.DataFrame(formatted_regex, columns=["regex", "scope", "type"])
    
    regex_df.drop_duplicates('regex', inplace=True)
    
    if neg_only:
        regex_df = regex_df[regex_df['type'].isin(['neg'])]
    
    return regex_df['regex']


0       pas de cause           
1       adéquat pour lécarter  
2       adéquat pour léliminer 
3       suffisant pour lécarter
4       suffisant pour lexclure
                 ...           
1256    toujours               
1257    alambic                
1258    immobile               
1260    silencieux             
1280    déjà                   
Name: regex, Length: 558, dtype: object